In [6]:
#Import the Libraries
import numpy as np
from scipy.io import loadmat  # this is the SciPy module that loads mat-files
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd
import os
from natsort import natsorted
import re

#Make a list of the files
Mat_Beh_dir='/data/projects/istart/Monetary_Incentive/data'
ML_EvFiles_list=natsorted([os.path.join(root,f) for root,dirs,files in os.walk(
    Mat_Beh_dir) for f in files if 'run' in f])

print("The number of Subjects are: %s"%(len([x for x in os.listdir(Mat_Beh_dir) if x.startswith('sub')])))
print("The number of files are: %s"%(len(ML_EvFiles_list)))
print("----------------------------------------")
#Make Conditions for Run1 and Run2
run_cond=loadmat('%s/../timing/run1.mat'%(Mat_Beh_dir))
run1=run_cond['run']['cond'][0,0]
run1 = np.where(run1==['1'], 'Large_gain', run1)
run1 = np.where(run1==['2'], 'Large_loss', run1)
run1 = np.where(run1==['3'], 'Small_gain', run1)
run1 = np.where(run1==['4'], 'Small_loss', run1)
run1 = np.where(run1==['5'], 'Neutral', run1)
run1=np.concatenate(run1)

run_cond=loadmat('%s/../timing/run2.mat'%(Mat_Beh_dir))
run2=run_cond['run']['cond'][0,0]
run2 = np.where(run2==[1], 'Large_gain', run2)
run2 = np.where(run2==['2'], 'Large_loss', run2)
run2 = np.where(run2==['3'], 'Small_gain', run2)
run2 = np.where(run2==['4'], 'Small_loss', run2)
run2 = np.where(run2==['5'], 'Neutral', run2)
run2=np.concatenate(run2)

#Load the .mat convert to dataframe add trial_type and detect bad data
ML_EvFiles_list=[f for f in ML_EvFiles_list if ('sub-1253' in f)&('run-1' in f)]

for ML_EvFile in ML_EvFiles_list:
    print(ML_EvFile)
    sub='sub-'+re.search('sub-(.*)/',ML_EvFile).group(1)
    run=re.search('run-(.*)_out',ML_EvFile).group(1)
    print(sub,run)
    mat = loadmat(ML_EvFile)  # load mat-file
    mdata = mat['output']  # variable in mat file
    mdtype = mdata.dtype  # dtypes of structures are "unsized objects"
    # * SciiencPy reads in structures as structured NumPy arrays of dtype object
    # * The size of the array is the size of the structure array, not the number
    #   elements in any particular field. The shape defaults to 2-dimensional.
    # * For convene make a dictionary of the data using the names from dtypes
    # * Since the structure has only one element, but is 2-D, index it at [0, 0]
    ndata = {n: mdata[n][0, 0] for n in mdtype.names}
    # Reconstruct the columns of the data table from just the time series
    # Use the number of intervals to test if a field is a column or metadata
    columns = [n for n in ndata]
    #Check fo frame
    # now make a data frame, setting the time stamps as the index
    lens=[len(x[0]) for x in ndata.values()]
    if lens.count(lens[0]) == len(lens):
        df = pd.DataFrame(np.transpose(np.concatenate([ndata[c] for c in columns])),
                  columns=columns)
        if run=='1':
            if len(df)==50:
                df['trial_type']=run1[0:50]
            elif len(df)==75:
                df['trial_type']=run1
        if run=='2':
            df['trial_type']=run2
        #print(df)
        tmp1=df[['target_starts','trial_starts','trial_type']]
        tmp1['duration']=tmp1['target_starts']-tmp1['trial_starts']
        tmp1['onset']=tmp1['trial_starts']
        tmp1=tmp1[['onset','duration','trial_type']]
        tmp2=df[['target_starts','outcome']]
        tmp2['onset']=tmp2['target_starts']+1.0
        tmp2['duration']=1.0
        map_dict = {1: "ConHit", 0 : "ConMiss"}
        tmp2["trial_type"] = tmp2["outcome"].map(map_dict)
        tmp2=tmp2[['onset','duration','trial_type']]
        df=pd.concat([tmp1,tmp2])
        if not os.path.isdir('../../istart-data/bids/%s/func'%(sub)):
            os.makedirs('../../istart-data/bids/%s/func'%(sub)) 
        df.to_csv('../../istart-data/bids/%s/func/%s_task-mid_run-%s_events.tsv'%(sub,sub,run),sep = '\t', index=False)
    else:
        print("Problem with %s run-%s"%(sub,run))

The number of Subjects are: 60
The number of files are: 83
----------------------------------------
/data/projects/istart/Monetary_Incentive/data/sub-1253/run-1_output.mat
sub-1253 1
Problem with sub-1253 run-1


<ipython-input-6-570021065cf9>:22: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  run1 = np.where(run1==['1'], 'Large_gain', run1)


In [14]:
print(ndata)
len(ndata['trial_starts'][0])


{'outcome': array([[1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0,
        1, 0, 1, 0, 0]], dtype=uint8), 'trial_starts': array([[  4.02052833,  10.10389897,  20.98718243,  30.67043727,
         39.15380236,  56.03718755,  63.32050107,  70.60396312,
         77.6872592 ,  88.57059367,  98.25392097, 109.13728351,
        115.22125519, 127.30400306, 133.38729733, 145.47064412,
        151.55401113, 158.83734881, 164.92066783, 174.60403768,
        181.88734608, 192.77072855, 202.45405162, 212.13742449,
        219.42067492, 227.90409042, 236.38805805, 244.87074241,
        259.354125  , 266.63744426, 273.72080554, 281.0041233 ,
        287.08748729, 294.17082714, 303.85415655, 312.33749175,
        325.62082157, 332.90423321, 340.18751021, 347.27085247,
        353.35420319, 364.23753193, 371.32091766, 382.20424308,
        397.88814121, 405.17094155, 420.85433496, 430.53760972,
        437.82125

50